In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
import os
load_dotenv() 
model=ChatOpenAI()

In [2]:
class BatsManState(TypedDict):
    runs:int
    balls:int
    four:int
    six:int
    sr:float
    bpb=float
    boundary_percentage:float

In [17]:
def calculate_SR(state:BatsManState)->BatsManState:
    runs=state["runs"]
    balls=state["balls"]
    sr=runs/balls*100
    # state["sr"]=sr

    return {'sr':sr}

def calculate_bpb(state:BatsManState)->BatsManState:
    runs=state["runs"]
    balls=state["balls"]
    fours=state["four"]
    six=state["six"]
    bpb=balls/(fours+six)
    # state["bpb"]=bpb
    return {"bpb":bpb}

def calculate_boundary_percent(state:BatsManState)->BatsManState:
    boundary_percentage=(((state["four"]*4)+(state["six"]*6))/state["runs"])*100
    # state["boundary_percentage"]=boundary_percentage
    return {"boundary_percentage":boundary_percentage}

def summary(state:BatsManState)->BatsManState:
    summary_info=f"""
    Strike Rate - {state['sr']} \n
    balls Per boundaet - {state["bpb"]} \n
    Boundary Percentage - {state["boundary_percentage"]}
    """
    # state["summary"]=summary_info
    return {"summary":summary_info}
    

In [18]:
graph=StateGraph(BatsManState)
graph.add_node("calculate_SR",calculate_SR)
graph.add_node("calculate_bpb",calculate_bpb)
graph.add_node("calculate_boundary_percent",calculate_boundary_percent)
graph.add_node("summary",summary)

graph.add_edge(START,'calculate_SR')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundary_percent')

graph.add_edge('calculate_SR','summary')
graph.add_edge('calculate_boundary_percent','summary')
graph.add_edge('calculate_bpb','summary')

graph.add_edge('summary',END)

workflow=graph.compile()


In [19]:
initial_state = {
    'runs': 100,
    'balls': 40,
    'four': 10,
    'six': 3
}


workflow.invoke(initial_state)

{'runs': 100,
 'balls': 40,
 'four': 10,
 'six': 3,
 'sr': 250.0,
 'bpb': 3.076923076923077,
 'boundary_percentage': 57.99999999999999,
 'summary': '\n    Strike Rate - 250.0 \n\n    balls Per boundaet - 3.076923076923077 \n\n    Boundary Percentage - 57.99999999999999\n    '}

## not parallel

In [13]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

class BatsManState(TypedDict):
    runs: int
    balls: int
    four: int
    six: int
    sr: float
    bpb: float
    boundary_percentage: float
    summary: str

def calculate_SR(state: BatsManState) -> BatsManState:
    runs = state["runs"]
    balls = state["balls"]
    sr = runs / balls * 100
    state["sr"] = round(sr, 2)
    return state

def calculate_bpb(state: BatsManState) -> BatsManState:
    runs = state["runs"]
    balls = state["balls"]
    fours = state["four"]
    six = state["six"]
    if (fours + six) > 0:
        bpb = balls / (fours + six)
    else:
        bpb = 0  # Handle division by zero
    state["bpb"] = round(bpb, 2)
    return state

def calculate_boundary_percent(state: BatsManState) -> BatsManState:
    runs = state["runs"]
    fours = state["four"]
    sixes = state["six"]
    if runs > 0:
        boundary_percentage = (((fours * 4) + (sixes * 6)) / runs) * 100
    else:
        boundary_percentage = 0  # Handle division by zero
    state["boundary_percentage"] = round(boundary_percentage, 2)
    return state

def summary(state: BatsManState) -> BatsManState:
    summary_info = f"""
    Strike Rate = {state['sr']}
    Balls Per Boundary = {state["bpb"]}
    Boundary Percentage = {state["boundary_percentage"]}%
    """
    state["summary"] = summary_info
    return state

# Build the graph with PROPER sequential execution
graph = StateGraph(BatsManState)
graph.add_node("calculate_SR", calculate_SR)
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percent", calculate_boundary_percent)
graph.add_node("summary", summary)

# Correct sequential execution
graph.add_edge(START, 'calculate_SR')
graph.add_edge('calculate_SR', 'calculate_bpb')
graph.add_edge('calculate_bpb', 'calculate_boundary_percent')
graph.add_edge('calculate_boundary_percent', 'summary')
graph.add_edge('summary', END)

workflow = graph.compile()

# Initial state - only provide the raw data
initial_state = {
    'runs': 100,
    'balls': 40,
    'four': 10,
    'six': 3,
    # Don't initialize calculated fields - let the nodes populate them
}

result = workflow.invoke(initial_state)
print("Final Result:")
print(result["summary"])
print("\nAll calculated values:")
print(f"Strike Rate: {result['sr']}")
print(f"Balls Per Boundary: {result['bpb']}")
print(f"Boundary Percentage: {result['boundary_percentage']}%")

Final Result:

    Strike Rate = 250.0
    Balls Per Boundary = 3.08
    Boundary Percentage = 58.0%
    

All calculated values:
Strike Rate: 250.0
Balls Per Boundary: 3.08
Boundary Percentage: 58.0%
